# news Crawling : url query 이용 
- url : http://media.daum.net -> [배열이력]
- url : https://news.daum.net/newsbox -> base url -> [특정 날짜] 
- url : https://news.daum.net/newsbox?regDate=20200505  -> [특정 페이지]
- url : https://news.daum.net/newsbox?regDate=20200505&page=2
- url : https://news.daum.net/newsbox?regDate=20200201&page=1

In [1]:
# 패키지설치
import urllib.request as req # url 요청
from bs4 import BeautifulSoup # html  파싱

# 1. url query 만들기
- date = 2020,2,1 ~ 2020.2.29
- page = 1~10

In [2]:
# base url + date
base_url = "https://news.daum.net/newsbox?regDate="
date = list(range(20200201, 20200230))
date # 20200201 ~ 20200229
len(date) # 29



# 20200201 ~ 20200229의 뉴스 url list 생성
url_list = [ base_url + str(d) for d in date]
# https://news.daum.net/newsbox?regDate=20200201



# base url + date + page
# 각 날짜의 뉴스별로 1~10page의 뉴스 url 생성
page = list(range(1, 11)) 
page # 1 ~ 10

pages = ['&page=' + str(p)   for p in page] 
pages # &page=1 ~ &page=10


final_url = []


for url in url_list : # base url + date = 29
    for page in pages : # page1 ~ page10 = 10
        final_url.append(url + page)
        # https://news.daum.net/newsbox?regDate=20200201&page=1

len(final_url) # 290 = 29 * 10
final_url[0]
# 'https://news.daum.net/newsbox?regDate=20200201&page=1'
final_url[-1]
# 'https://news.daum.net/newsbox?regDate=20200229&page=10'


'https://news.daum.net/newsbox?regDate=20200229&page=10'

# 2. Crawlier 함수 정의 

In [3]:
def Crawlier(url) : 
    # 1. url 요청
    res = req.urlopen(url)
    src = res.read() # source
       
    # 2. html 파싱
    src = src.decode('utf-8')
    html = BeautifulSoup(src, 'html.parser')
        
    # 3. tag[속성='값'] -> 'a[class="link_txt"]'
    links = html.select('a[class="link_txt"]')   
    
    
    one_page_data = [] # 빈 list
    cnt = 0 
    
    for link in links :
        link_str = str(link.string) # 내용 추출
        cnt += 1
        # print('cnt :', cnt) 
        # print('news :', link_str)
        one_page_data.append(link_str.strip()) # 문장 끝 불용어 처리(\n, 공백)
    
    return one_page_data[:40] # list

# 1page만 crawling
one_page_data = Crawlier(final_url[0])  # 2020. 02. 01 -> page1  
len(one_page_data)  # 134 -> 40 
type(one_page_data)  # list  



# 해당 기사 추출 
print(one_page_data[:10])

['의협 "감염병 위기경보 상향 제안..환자 혐오 멈춰야"', "'신종코로나 진원' 중국 후베이성 춘제연휴 13일까지 재연장", '높아진 국가청렴도..제도개혁이 관건', '"독일 공군기, 중국 우한서 100여명 태우고 본국으로"', '이견 여전한 방위비 협상..이달 타결될까', '의사협회 "감염위기 \'심각\'으로 상향을..위험국가 입국 제한도"', '6년 만의 시진핑 방한에도 영향.."3~4월은 어렵다"', '진천군, 우한 교민 임시생활 국가인재개발원 일대 방역 강화(종합)', "무사증 제주입국 중국인, 귀국 후 신종코로나 확진..'방역비상'", '31년 한우물 판 \'젊은 노포\' "전통만큼 중요한 건 변화" [\'창간 31\' - 세계일보와 \'동갑내기\' 가게들]']


# 3. 전체 기사 수집 : crwler 함수 호출

In [4]:
month_news = []
page_cnt = 0

# Crwler 함수 호출
for url in final_url:
    page_cnt += 1
    # print('page :', page_cnt)
    try : # url 없는 경우
        one_page_news = Crawlier(url) # 1page news
        # print('one page news')
        # print(one_page_news)
        
        # month_news.append(one_page_news) # [[1page],[2page],...]
        month_news.extend(one_page_news) # [1page ~ 299page]
    except :
        print('해당 url 없음 :',url)
        


len(final_url)
len(month_news) # 29dayx10pagex40news = 11600
print(month_news[:10])

['의협 "감염병 위기경보 상향 제안..환자 혐오 멈춰야"', "'신종코로나 진원' 중국 후베이성 춘제연휴 13일까지 재연장", '높아진 국가청렴도..제도개혁이 관건', '"독일 공군기, 중국 우한서 100여명 태우고 본국으로"', '이견 여전한 방위비 협상..이달 타결될까', '의사협회 "감염위기 \'심각\'으로 상향을..위험국가 입국 제한도"', '6년 만의 시진핑 방한에도 영향.."3~4월은 어렵다"', '진천군, 우한 교민 임시생활 국가인재개발원 일대 방역 강화(종합)', "무사증 제주입국 중국인, 귀국 후 신종코로나 확진..'방역비상'", '31년 한우물 판 \'젊은 노포\' "전통만큼 중요한 건 변화" [\'창간 31\' - 세계일보와 \'동갑내기\' 가게들]']


# 4. Binary file save

In [ ]:
import pickle # list -> file save -> load(list) : 리스트 객체 유지 가능

# file save
file = open('data/news_data.pck', mode='wb')
pickle.dump(month_news, file) 
file.close()

# file load
file = open('data/news_data.pck', mode='rb')
month_news_crawling2 = pickle.load(file)
month_news_crawling2
file.close()